In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
#load train and test data
train=pd.read_csv('/kaggle/input/digit-recognizer/train.csv', header='infer')
test=pd.read_csv('/kaggle/input/digit-recognizer/test.csv', header='infer')

In [3]:
#feature and label are defined
X_train=train.drop(columns=['label']).values
X_test=test.values 

y_train=train["label"].values

In [4]:
#convert feature into 0to1 range
X_train=(X_train)/255
X_test=(X_test)/255

#one-encoding perform in labels
y_train=to_categorical(y_train)

In [5]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=0.2, random_state=2)

In [6]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(33600, 28, 28, 1) (33600, 10)
(8400, 28, 28, 1) (8400, 10)


In [7]:
#define a model
model=tf.keras.Sequential()

#Convolution layer 1
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False,input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))

#Convolution layer 2
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#Convolution layer 3
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

#Convolution layer 4
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding='valid', activation=None, use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))

#fully connected layer
model.add(Flatten())
#dense layer 1
model.add(Dense(units=200))
model.add(BatchNormalization())
model.add(Activation('relu'))
#dense layer 2
model.add(Dense(units=100))
model.add(BatchNormalization())
model.add(Activation('relu'))
#output layer
model.add(Dense(units=10))
model.add(BatchNormalization())
model.add(Activation('softmax'))

2022-03-11 14:11:34.162199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 14:11:34.326366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 14:11:34.327423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 14:11:34.329460: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [8]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
tr_generator = ImageDataGenerator(featurewise_center=False,
                             samplewise_center=False,
                             featurewise_std_normalization=False,
                             samplewise_std_normalization=False,
                             zca_whitening=False,
                             rotation_range=10,
                             zoom_range=0.1,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False)
te_generator = ImageDataGenerator()

train_generator = tr_generator.flow(X_train, y_train, batch_size=100, shuffle=True)
test_generator = te_generator.flow(X_val, y_val, batch_size=100, shuffle=True)



In [10]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(33600, 28, 28, 1)
(8400, 28, 28, 1)
(33600, 10)
(8400, 10)


In [11]:
#reduceLROnPlateau=ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [12]:
model.fit(train_generator, epochs=20,batch_size=100,validation_data=test_generator)

2022-03-11 14:11:37.610512: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-03-11 14:11:39.858681: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


336/336 [==============================] - 22s 41ms/step - loss: 0.4319 - accuracy: 0.9431 - val_loss: 0.8081 - val_accuracy: 0.8273
Epoch 2/20
336/336 [==============================] - 14s 42ms/step - loss: 0.1955 - accuracy: 0.9810 - val_loss: 0.1583 - val_accuracy: 0.9856
Epoch 3/20
336/336 [==============================] - 14s 41ms/step - loss: 0.1337 - accuracy: 0.9847 - val_loss: 0.0887 - val_accuracy: 0.9918
Epoch 4/20
336/336 [==============================] - 14s 41ms/step - loss: 0.1016 - accuracy: 0.9862 - val_loss: 0.0786 - val_accuracy: 0.9889
Epoch 5/20
336/336 [==============================] - 14s 40ms/step - loss: 0.0789 - accuracy: 0.9882 - val_loss: 0.0742 - val_accuracy: 0.9877
Epoch 6/20
336/336 [==============================] - 13s 39ms/step - loss: 0.0678 - accuracy: 0.9883 - val_loss: 0.0505 - val_accuracy: 0.9905
Epoch 7/20
336/336 [==============================] - 15s 44ms/step - loss: 0.0623 - accuracy: 0.9880 - val_loss: 0.1384 - val_accuracy: 0.9648
Epo

In [13]:
#prediction on test images
predictions=model.predict(X_test) 
#choose max probabality class and assign the class 
predictions=np.argmax(predictions,axis=1) 

#read the sample_submission csv file 
sub=pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv', header='infer')
#assign labale value
sub["Label"]=predictions
sub.to_csv('submission.csv', index=False)